In [15]:
import os
import cv2
import json
import torch
import torchvision
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torch.optim as optim
from PIL import Image, ImageDraw
from pathlib import Path
import pandas as pd
# import Mask R-CNN
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

new_size = (600,800)
transform = T.Compose([T.Resize(new_size),T.ToTensor()])
image_dir = "/content/drive/MyDrive/Ara_proje_dataset/Furkan_Pan/images"
json_dir = "/content/drive/MyDrive/Ara_proje_dataset/Furkan_Pan/JSON/outputs"

In [19]:
# The name for the project
project_name = f"pytorch-mask-r-cnn-instance-segmentation"

# The path for the project folder
project_dir = Path(f"./{project_name}/")

# Create the project directory if it does not already exist
project_dir.mkdir(parents=True, exist_ok=True)

# Define path to store datasets
dataset_dir = Path("/content/drive/MyDrive/Ara_proje_dataset")

pd.Series({
    "Project Directory:": project_dir,
    "Dataset Directory:": dataset_dir
}).to_frame().style.hide(axis='columns')

Project Directory:,pytorch-mask-r-cnn-instance-segmentation
Dataset Directory:,/content/drive/MyDrive/Ara_proje_dataset


In [36]:
imagess = list(sorted(os.listdir(image_dir)))
json_filess = list(sorted(os.listdir(json_dir)))

not_found = []
for file in imagess:
  img_name = Path(file).stem
  found=0
  i=0
  while i < len(json_filess) and found==0:
    json_name = Path(json_filess[i]).stem
    if img_name == json_name:
      found=1
    i+=1
  if found==0 :
    not_found.append(img_name)

not_found

[]

In [37]:
# Display the names of the folders using a Pandas DataFrame
pd.DataFrame({"Image File": [file for file in imagess], "json_files" : [file for file in json_filess]})

,Image File,json_files
0,1.jpg,1.json
1,100.jpg,100.json
2,102.jpg,102.json
3,104.jpg,104.json
4,105.jpg,105.json
...,...,...
745,963.jpg,963.json
746,964.jpg,964.json
747,965.jpg,965.json
748,98.jpg,98.json


In [13]:
class RadiographDataset(Dataset):
  def __init__(self, image_dir, json_dir, transform=None):
    self.image_dir = image_dir
    self.json_dir = json_dir
    self.transform = transform
    self.images = list(sorted(os.listdir(image_dir)))
    self.json_files = list(sorted(os.listdir(json_dir)))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_path = os.path.join(self.image_dir, self.images[idx])
    json_path = os.path.join(self.json_dir, self.json_files[idx])
    image = Image.open(img_path)  # Load image

    annotations = self.get_data_from_json(json_path) # Load data from json file

    flattened_polygon_points = self.flatten_polygon_points(annotations)

        # Create mask from polygon
    mask = self.create_mask_from_polygon(image.size, flattened_polygon_points)

        # Apply transformations if any
    if self.transform is not None:
      image = self.transform(image)
      mask = self.transform(mask)
      return image, mask

  @staticmethod
  def get_data_from_json(json_file_path):
    json_file = open(json_file_path,'r')
    data = json.load(json_file)
    return data

  @staticmethod
  def create_mask_from_polygon(size, polygon_coord):
    img_mask = Image.new('L', size, 0)
    draw = ImageDraw(img_mask)
    draw.polygon(polygon_coord, outline=1, fill=1)
    return img_mask

  @staticmethod
  def flatten_polygon_points(annotations):
    xy = int(len(annotations["outputs"]["object"][0]["polygon"])/2) +1
    polygon_coordinates = []
    for i in range(1,xy):
      x = annotations["outputs"]["object"][0]["polygon"]["x" + str(i)]
      y = annotations["outputs"]["object"][0]["polygon"]["y" + str(i)]
      point = (x, y)
      polygon_coordinates.append(point)
      return polygon_coordinates

In [ ]:
dataset = RadiographDataset(image_dir, json_dir, transforms=transform)

# define the model
class_num = 2
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.005)

data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

